In [ ]:
https://aclanthology.org/2020.acl-main.90.pdf

https://medium.com/@waqarhaider1292/chatbot-by-huggingface-document-based-question-answering-c4fd94ec35db

https://medium.com/@lokaregns/question-answering-with-hugging-face-transformers-a-beginners-guide-487ae1a91b9a

In [ ]:
# from transformers import pipeline

# # Load the question answering pipeline
# model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
# qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

# # Initialize conversation history dictionary
# conversation_history = {
#     "context": None,
#     "questions": [],
#     "answers": []
# }

# def answer_question(context, question):
#     # Use the QA pipeline to answer the question based on the context
#     try:
#         result = qa_pipeline(question=question, context=context)
#         return result
#     except Exception as e:
#         print("Error occurred while answering the question:", e)
#         return "Sorry, I couldn't find an answer to that question."

# def detect_follow_up(question):
#     # Simple algorithm to detect follow-up questions
#     keywords = ["next", "then", "after", "following"]
#     for keyword in keywords:
#         if keyword in question.lower():
#             return True
#     return False

# def add_to_conversation_history(context, question, answer):
#     # Add the current context, question, and answer to conversation history
#     conversation_history["context"] = context
#     conversation_history["questions"].append(question)
#     conversation_history["answers"].append(answer)

# def answer_with_history(question):
#     # Check if the current question is a follow-up question
#     is_follow_up = detect_follow_up(question)
    
#     if is_follow_up:
#         # If it's a follow-up question, use the context and previous answers to answer
#         context = conversation_history["context"]
#         questions = conversation_history["questions"]
#         answers = conversation_history["answers"]
#         for i, prev_question in enumerate(reversed(questions)):
#             if detect_follow_up(prev_question):
#                 continue
#             context = context.replace(answers[-(i+1)], "")  # Remove previous answer from context
#             break
#         return answer_question(context, question)
#     else:
#         # If it's not a follow-up question, answer it directly
#         return answer_question(conversation_history["context"], question)

# # Example context
# context = "The Apollo program, also known as Project Apollo, was the third United States human spaceflight program carried out by the National Aeronautics and Space Administration (NASA), which accomplished landing the first humans on the Moon from 1969 to 1972. First conceived during the Eisenhower administration as a three-man spacecraft to follow the one-man Project Mercury which put the first Americans in space, Apollo was later dedicated to President John F. Kennedy's national goal for the 1960s of 'landing a man on the Moon and returning him safely to the Earth'."

# # Example questions
# questions = [
#     "What was the Apollo program?",
#     "after Who carried out the Apollo program?",
#     "When did the Apollo program take place?",
#     "What happened after the Apollo program?",
#     "How was the Apollo program followed?"
# ]

# # Answer the questions with conversation history tracking
# for question in questions:
#     answer = answer_with_history(question)
#     add_to_conversation_history(context, question, answer)
#     print("Question:", question)
#     print("Answer:", answer)
#     print()


In [52]:
import PySimpleGUI as sg
from transformers import pipeline

# Load the question answering pipeline
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

# Initialize conversation history dictionary
conversation_history = {
    "context": None,
    "questions": [],
    "answers": []
}

def answer_question(context, question):
    try:
        if len(conversation_history['questions']) != 0:
            concatenated_text = ""
            for history_question, answer in zip(conversation_history["questions"], conversation_history["answers"]):
                concatenated_text += f"{history_question} {answer}. "
            
            question = concatenated_text + question


        result = qa_pipeline(question=question, context=context)
        return question, result['answer'], result['score']
    except Exception as e:
        print("Error occurred while answering the question:", e)
        return "Sorry, I couldn't find an answer to that question."



def add_to_conversation_history(context, question, answer):
    # Add the current context, question, and answer to conversation history
    conversation_history["context"] = context
    conversation_history["questions"].append(question)
    conversation_history["answers"].append(answer)

# def answer_with_history(question):

#         # If it's a follow-up question, use the context and previous answers to answer
#         context = conversation_history["context"]
#         questions = conversation_history["questions"]
#         answers = conversation_history["answers"]
#         return answer_question(context, question)
#         # If it's not a follow-up question, answer it directly

# Define GUI layout
layout = [
    [sg.Text("Question-Answering System")],
    [sg.Text("Context:"), sg.Multiline(key='-CONTEXT-', size=(60, 10))],
    [sg.Multiline(size=(60, 10), key='-OUTPUT-', disabled=True)],
    [sg.InputText(key='-INPUT-')],
    [sg.Button('Ask'), sg.Button('Exit')],
]

# Create the GUI window
window = sg.Window('QA System', layout)

# Main event loop
while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break

    user_input = values['-INPUT-']
    context = values['-CONTEXT-']

    if user_input:
        question = user_input

        # Get response to the user's question
        concat, answer, score = answer_question(context, question)
        add_to_conversation_history(context, question, answer)


        # Display the answer in the GUI window
        window['-OUTPUT-'].print("Question: " + concat)
        window['-OUTPUT-'].print("Answer: " + answer)
        window['-OUTPUT-'].print("Score: " + str(score))
        window['-OUTPUT-'].print("")

window.close()


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [36]:
import spacy

# Load English language model
nlp = spacy.load("en_core_web_md")

# Define two example sentences
sentence1 = "who is b? Tong."
sentence2 = "cat"

# Process the sentences with spaCy
doc1 = nlp(sentence1)
doc2 = nlp(sentence2)

# Calculate cosine similarity between the document vectors
similarity = doc1.similarity(doc2)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.12360134012040346


In [51]:
import nltk
# from nltk.translate.bleu_score import sentence_bleu
# from datasets import load_metric
# from nltk.tokenize import word_tokenize
# bleu = load_metric("bleu")

# # Reference answer
# reference = "The cat is on the mat."

# # Generated answer
# generated = "There is a cat on the mat."

# # Tokenize reference and generated answers
# reference_tokens = word_tokenize(reference.lower())
# generated_tokens = word_tokenize(generated.lower())

# # Calculate BLEU score
# bleu_score = bleu.compute(predictions=[reference_tokens], references=[generated_tokens])
# print("BLEU Score:", bleu_score)

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ValueError: Got a string but expected a list instead: 'there'